In [84]:
import importlib
import pandas as pd
import numpy as np
import os
import py_vollib_vectorized
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [93]:
import classes, utils, iv_models
_ = importlib.reload(classes)
_ = importlib.reload(utils)
_ = importlib.reload(iv_models)


from classes import Ticker
from utils import create_market_state
from iv_models import BasicMidIVPolynomial, MidIV

In [94]:
t = Ticker("AXISBANK", expiry_date="20210624")
oc1 = t.get_option_chain('20210623 09:30:00')
oc2 = t.get_option_chain('20210623 11:30:00')
oc1.fit_iv_model(MidIV())
oc2.fit_iv_model(MidIV())

loading data from ./data/20210624/influxOP_AXISBANK21JUN_20210624.csv
loading data from ./data/20210623/influxOP_AXISBANK21JUN_20210623.csv


In [111]:
res1 = oc1.ivs_prices_greeks([74000, 75000, 76000])
res1[['delta', 'vega', 'gamma', 'theta', 'fit_iv', 'fit_price']]

,delta,vega,gamma,theta,fit_iv,fit_price
0,-0.426875,18.258711,0.000370,-183.994797,0.227586,334.998764
1,0.235907,14.330904,0.000277,-125.493105,0.238758,152.494934
2,0.089938,7.552664,0.000122,-53.150896,0.284916,54.893986


4.155427001646

In [108]:
res2 = oc2.ivs_prices_greeks([74000, 75000, 76000])
res2[['delta', 'vega', 'fit_iv', 'fit_price']]

,delta,vega,fit_iv,fit_price
0,-0.388038,17.328387,0.215200,267.494164
1,0.241378,14.099732,0.226588,144.980338
2,0.078664,6.636267,0.265840,42.464180


In [117]:
oc1.df[oc1.df['strike'] == 74000]

,level_0,strike,askPrice_1,bidPrice_1,futPrice,spread,time_to_expiry,moneyness,instrument,bid_iv,ask_iv,mid_iv,spread_iv
470,2021-06-23 09:30:00,74000,340.0,330.0,74187.5,10.0,0.003938,-0.002531,p,0.224848,0.230325,0.227586,0.005477


In [118]:
oc2.df[oc2.df['strike'] == 74000]

,level_0,strike,askPrice_1,bidPrice_1,futPrice,spread,time_to_expiry,moneyness,instrument,bid_iv,ask_iv,mid_iv,spread_iv
5681,2021-06-23 11:30:00,74000,275.0,260.0,74270.0,15.0,0.003709,-0.003642,p,0.210872,0.219528,0.2152,0.008656


In [122]:
334.998764 + 18.258711 * (0.2152 - 0.227586) * 100 - 35.217181  -0.042049 * 365

261.81845855539996

In [11]:
oc2.iv_parameters()

array([29.84387812, -0.11291487,  0.31478401])

In [12]:
oc1.iv_parameters()

array([26.76851682, -0.12406091,  0.33292242])

In [121]:
-0.042049 * 365

-15.347885000000002

In [100]:
delta_diff = res1['delta'] * (oc2.future_price() - oc1.future_price()) 
gamma_diff = res1['gamma'] * (res2['delta'] - res1['delta']) 
theta_diff = res1['theta'] * (oc1.time_to_expiry() - oc2.time_to_expiry())

print("delta_diff", delta_diff)
print("gamma_diff", gamma_diff)
print("theta_diff", theta_diff)

delta_diff 0   -35.217181
1    19.462351
2     7.419909
Name: delta, dtype: float64
gamma_diff 0    0.000014
1    0.000002
2   -0.000001
dtype: float64
theta_diff 0   -0.042049
1   -0.028679
2   -0.012147
Name: theta, dtype: float64


In [101]:
s = 0

for i in range(len(oc1.iv_parameters())):
    s += res1['vega'] * res1[f'greek_{i}'] * (oc2.iv_parameters()[i] - oc1.iv_parameters()[i])
s

0    1.167314
1   -0.337600
2   -0.440532
dtype: float64

In [15]:
8.010609 * (1.0 * (0.33292242 - 0.31478401) + 0.028021 * (-0.12406091 + 0.11291487) )

0.14279781146028075